<a href="https://colab.research.google.com/github/maurya-anuj/Eip4/blob/master/Assign5/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/Deep Vision/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%tensorflow_version 1.x

import cv2
# import json

import numpy as np
import pandas as pd

# from functools import partial
# from pathlib import Path 
# from tqdm import tqdm

from google.colab.patches import cv2_imshow

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# from keras.applications import VGG16
from keras.layers.core import Dropout
# from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
# from keras.optimizers import SGD
# from keras.preprocessing.image import ImageDataGenerator


batch = 64

#########################
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]


##########  AUGUMENTATION GEN.... CUTOUT  #####################

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    def __init__(self, df, batch_size=32, shuffle=True, augumentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augumentation = augumentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        # image = image/255
        if self.augumentation is not None:
            image = self.augumentation.flow(image, shuffle=False, batch_size=batch).next()
        image = image/255
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
print(train_df.shape, val_df.shape)

########################################################
pixel_level = False
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.1, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

########################################

from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
        # # set input mean to 0 over the dataset
        # featurewise_center=False,
        # # set each sample mean to 0
        # samplewise_center=False,
        # # divide inputs by std of dataset
        # featurewise_std_normalization=False,
        # # divide each input by its std
        # samplewise_std_normalization=False,
        # # apply ZCA whitening
        # zca_whitening=False,
        # # epsilon for ZCA whitening
        # zca_epsilon=1e-06,
        # # randomly rotate images in the range (deg 0 to 180)
        # rotation_range=10,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # # set range for random shear
        # shear_range=0.,
        # # set range for random zoom
        # zoom_range=0.1,
        # # set range for random channel shifts
        # channel_shift_range=0.,
        # # set mode for filling points outside the input boundaries
        # fill_mode='nearest',
        # # value used for fill_mode = "constant"
        # cval=0.,
        # # randomly flip images
        # horizontal_flip=True,
        # # randomly flip images
        # vertical_flip=False,
        # # set rescaling factor (applied before any other transformation)
        # rescale=1.0/255.0,
        # set function that will be applied on each input
        preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=pixel_level),
        # # image data format, either "channels_first" or "channels_last"
        # data_format=None,
        # # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=batch, augumentation = aug)
valid_gen = PersonDataGenerator(val_df, batch_size=batch*2, shuffle=False, augumentation = None)  # val_df

# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}

#########################################################################################
from keras.models import model_from_json
json_file = open("/content/gdrive/My Drive/Deep Vision/resnet34_new.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.layers.pop() # Remove last dense layer

neck = model.output

def build_tower(in_layer):
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck

def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="sigmoid", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))

model = Model(
    inputs=model.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

####################  COMPILE  ############################
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
# Prepare callbacks for model saving and for learning rate adjustment.

# Prepare model model saving directory.
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

model_save = keras.callbacks.ModelCheckpoint('model{epoch:08d}.h5', period=5) 

lr_reducer = ReduceLROnPlateau(factor=.05,
                               monitor='val_loss',
                               cooldown=0,
                               patience=5,
                               verbose=1,
                               min_delta=0.01,
                               min_lr=0.5e-7)
def lr_schedule(epoch):
    lr = 1.4
    if epoch > 60:
        lr *= 1e-1
    elif epoch > 30:
        lr *= 1e-1
    elif epoch > 25:
        lr *= 1e-1
    elif epoch > 15:
        lr *= 0.5e-1
    elif epoch > 10:
        lr *= 1e-1
    elif epoch > 8:
        lr *= 0.5
    elif epoch > 5:
        lr = 0.9
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [lr_scheduler, model_save]

from keras.optimizers import SGD
opt = SGD(lr=0.1)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

######################  FIT  #######################################
model_info = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1,
    callbacks = callbacks
)

Using TensorFlow backend.


(11537, 28) (2036, 28)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
Learning rate:  1.4
180/180 [==============================] - 165s 917ms/step - loss: 9.4844 - gender_output_loss: 0.6834 - image_quality_output_loss: 0.9824 - age_output_loss: 1.4297 - weight_output_loss: 0.9927 - bag_output_loss: 0.9177 - footwear_output_loss: 1.0138 - pose_output_loss: 0.9324 - emotion_output_loss: 0.9120 - gender_output_acc: 0.5604 - image_quality_output_acc: 0.5520 - age_output_acc: 0.4038 - weight_output_acc: 0.6322 - bag_output_acc: 0.5657 - footwear_output_acc: 0.4851 - pose_output_acc: 0.6174 - emotion_output_acc: 0.7126 - val_loss: 9.6163 - val_gender_output_loss: 0.6903 - val_image_quality_output_loss: 0.9960 - val_age_output_loss: 1.4621 - val_weight_output_loss: 0.9982 - val_bag_output_loss: 0.9398

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 30 could not be retrieved. It could be because a worker has died.
  UserWarning)


154/180 [========================>.....] - ETA: 24s - loss: 6.3592 - gender_output_loss: 0.1443 - image_quality_output_loss: 0.9701 - age_output_loss: 1.3058 - weight_output_loss: 0.9116 - bag_output_loss: 0.8001 - footwear_output_loss: 0.6312 - pose_output_loss: 0.2298 - emotion_output_loss: 0.8334 - gender_output_acc: 0.9473 - image_quality_output_acc: 0.5501 - age_output_acc: 0.4316 - weight_output_acc: 0.6507 - bag_output_acc: 0.6475 - footwear_output_acc: 0.7285 - pose_output_acc: 0.9220 - emotion_output_acc: 0.7169

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 16 could not be retrieved. It could be because a worker has died.
  UserWarning)


172/180 [===========================>..] - ETA: 8s - loss: 6.3640 - gender_output_loss: 0.1412 - image_quality_output_loss: 0.9686 - age_output_loss: 1.3063 - weight_output_loss: 0.9109 - bag_output_loss: 0.8001 - footwear_output_loss: 0.6336 - pose_output_loss: 0.2329 - emotion_output_loss: 0.8375 - gender_output_acc: 0.9489 - image_quality_output_acc: 0.5515 - age_output_acc: 0.4306 - weight_output_acc: 0.6503 - bag_output_acc: 0.6459 - footwear_output_acc: 0.7277 - pose_output_acc: 0.9214 - emotion_output_acc: 0.7159

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 62 could not be retrieved. It could be because a worker has died.
  UserWarning)


180/180 [==============================] - 227s 1s/step - loss: 6.3617 - gender_output_loss: 0.1396 - image_quality_output_loss: 0.9678 - age_output_loss: 1.3090 - weight_output_loss: 0.9111 - bag_output_loss: 0.8002 - footwear_output_loss: 0.6322 - pose_output_loss: 0.2325 - emotion_output_loss: 0.8364 - gender_output_acc: 0.9493 - image_quality_output_acc: 0.5529 - age_output_acc: 0.4284 - weight_output_acc: 0.6495 - bag_output_acc: 0.6455 - footwear_output_acc: 0.7282 - pose_output_acc: 0.9214 - emotion_output_acc: 0.7163 - val_loss: 8.1226 - val_gender_output_loss: 0.4276 - val_image_quality_output_loss: 0.9784 - val_age_output_loss: 1.4698 - val_weight_output_loss: 0.9825 - val_bag_output_loss: 0.8760 - val_footwear_output_loss: 0.9125 - val_pose_output_loss: 0.9838 - val_emotion_output_loss: 0.9592 - val_gender_output_acc: 0.8578 - val_image_quality_output_acc: 0.5557 - val_age_output_acc: 0.3740 - val_weight_output_acc: 0.6432 - val_bag_output_acc: 0.5953 - val_footwear_output_a

Process ForkPoolWorker-927:
Process ForkPoolWorker-930:
Process ForkPoolWorker-925:
Process ForkPoolWorker-928:
Process ForkPoolWorker-926:
Process ForkPoolWorker-929:
Process ForkPoolWorker-932:
Process ForkPoolWorker-935:
Process ForkPoolWorker-917:
Process ForkPoolWorker-933:
Process ForkPoolWorker-934:
Process ForkPoolWorker-931:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6

KeyboardInterrupt: ignored

In [3]:
Score = model.evaluate_generator(valid_gen)
print([[a, b] for a,b in zip(df.columns,Score[9:])])

[['gender', 0.8578125], ['imagequality', 0.5557291666666667], ['age', 0.37395833333333334], ['weight', 0.6432291666666666], ['carryingbag', 0.5953125], ['footwear', 0.6135416666666667], ['emotion', 0.6890625], ['bodypose', 0.6880208333333333]]


In [4]:
def lr_schedule(epoch):
    lr = 0.01
    if epoch > 60:
        lr *= 1e-6
    elif epoch > 30:
        lr *= 1e-5
    elif epoch > 25:
        lr *= 1e-4
    elif epoch > 15:
        lr *= 0.5e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 8:
        lr *= 0.5e-1
    elif epoch > 5:
        lr *= 0.5
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [lr_scheduler, model_save]

from keras.optimizers import SGD
opt = SGD(lr=0.1)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

######################  FIT  #######################################
model_info = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1,
    callbacks = callbacks
)

Epoch 1/100
Learning rate:  0.01
180/180 [==============================] - 163s 907ms/step - loss: 6.1422 - gender_output_loss: 0.0924 - image_quality_output_loss: 0.9667 - age_output_loss: 1.2750 - weight_output_loss: 0.8876 - bag_output_loss: 0.7932 - footwear_output_loss: 0.5826 - pose_output_loss: 0.1794 - emotion_output_loss: 0.8328 - gender_output_acc: 0.9698 - image_quality_output_acc: 0.5554 - age_output_acc: 0.4401 - weight_output_acc: 0.6578 - bag_output_acc: 0.6445 - footwear_output_acc: 0.7476 - pose_output_acc: 0.9441 - emotion_output_acc: 0.7163 - val_loss: 7.2837 - val_gender_output_loss: 0.3653 - val_image_quality_output_loss: 0.9720 - val_age_output_loss: 1.3922 - val_weight_output_loss: 0.9369 - val_bag_output_loss: 0.8558 - val_footwear_output_loss: 0.8282 - val_pose_output_loss: 0.5006 - val_emotion_output_loss: 0.9003 - val_gender_output_acc: 0.8802 - val_image_quality_output_acc: 0.5573 - val_age_output_acc: 0.3839 - val_weight_output_acc: 0.6427 - val_bag_output

Process ForkPoolWorker-1165:
Process ForkPoolWorker-1164:
Process ForkPoolWorker-1168:
Process ForkPoolWorker-1166:
Process ForkPoolWorker-1167:
Process ForkPoolWorker-1159:
Process ForkPoolWorker-1162:
Process ForkPoolWorker-1169:
Process ForkPoolWorker-1163:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1158:
Process ForkPoolWorker-1160:
Process ForkPoolWorker-1161:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/

KeyboardInterrupt: ignored

In [5]:
Score = model.evaluate_generator(valid_gen)
print([[a, b] for a,b in zip(df.columns,Score[9:])])

[['gender', 0.8807291666666667], ['imagequality', 0.5583333333333333], ['age', 0.3875], ['weight', 0.6463541666666667], ['carryingbag', 0.5984375], ['footwear', 0.6510416666666666], ['emotion', 0.8411458333333334], ['bodypose', 0.6880208333333333]]
